# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

np.random.seed(1)
tf.random.set_seed(1)

## Get the data

In [2]:
data = pd.read_csv("./data/basketball.csv")

In [3]:
data.shape

(1230, 37)

In [4]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [5]:
y = data['W']
X = data.drop('W', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Data Transformation

In [7]:
#Target variables need to be an array with integer type
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [8]:
#Check the first 10 values of the train_y data set
y_train[0:10]

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 0], dtype=int32)

In [9]:
#Convert input variables to a 2-D array with float data type
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
X_train

array([[  1.,   2.,   7., ...,   6.,   2.,   4.],
       [  2.,   2.,   1., ...,  18.,  23.,  21.],
       [ -4.,  -4.,  -6., ..., -14., -14., -16.],
       ...,
       [ -3.,  -5.,  -2., ...,  14.,  13.,   9.],
       [ -2.,  -2.,  -2., ...,  13.,  15.,  12.],
       [  0.,  -5.,  -9., ...,  -3.,  -3.,  -5.]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [12]:
X_train.shape, y_train.shape

((861, 36, 1), (861,))

In [13]:
X_train

array([[[  1.],
        [  2.],
        [  7.],
        ...,
        [  6.],
        [  2.],
        [  4.]],

       [[  2.],
        [  2.],
        [  1.],
        ...,
        [ 18.],
        [ 23.],
        [ 21.]],

       [[ -4.],
        [ -4.],
        [ -6.],
        ...,
        [-14.],
        [-14.],
        [-16.]],

       ...,

       [[ -3.],
        [ -5.],
        [ -2.],
        ...,
        [ 14.],
        [ 13.],
        [  9.]],

       [[ -2.],
        [ -2.],
        [ -2.],
        ...,
        [ 13.],
        [ 15.],
        [ 12.]],

       [[  0.],
        [ -5.],
        [ -9.],
        ...,
        [ -3.],
        [ -3.],
        [ -5.]]], dtype=float32)

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [14]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_test, y_test))

Epoch 1/50


2023-04-16 17:28:54.037006: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


27/27 [==============================] - 0s 4ms/step - loss: 0.7840 - accuracy: 0.7271 - val_loss: 0.4976 - val_accuracy: 0.7751
Epoch 2/50
27/27 [==============================] - 0s 1ms/step - loss: 0.5022 - accuracy: 0.7689 - val_loss: 0.4337 - val_accuracy: 0.8076
Epoch 3/50
27/27 [==============================] - 0s 1ms/step - loss: 0.4814 - accuracy: 0.7758 - val_loss: 0.4125 - val_accuracy: 0.8211
Epoch 4/50
27/27 [==============================] - 0s 1ms/step - loss: 0.4306 - accuracy: 0.8002 - val_loss: 0.4192 - val_accuracy: 0.7995
Epoch 5/50
27/27 [==============================] - 0s 1ms/step - loss: 0.4020 - accuracy: 0.8188 - val_loss: 0.4710 - val_accuracy: 0.7805
Epoch 6/50
27/27 [==============================] - 0s 1ms/step - loss: 0.3960 - accuracy: 0.8165 - val_loss: 0.4073 - val_accuracy: 0.8157
Epoch 7/50
27/27 [==============================] - 0s 1ms/step - loss: 0.3828 - accuracy: 0.8177 - val_loss: 0.4315 - val_accuracy: 0.7967
Epoch 8/50
27/27 [=============

In [16]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.6983422040939331, 0.7696477174758911]

In [17]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.70
accuracy: 76.96%


# Simple RNN with one layer

In [18]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train, 
    y_train, 
    epochs=50,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop]
)

Epoch 1/50
27/27 [==============================] - 1s 7ms/step - loss: 0.4378 - accuracy: 0.7816 - val_loss: 0.4035 - val_accuracy: 0.8184
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4122 - accuracy: 0.7991 - val_loss: 0.3996 - val_accuracy: 0.8266
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4110 - accuracy: 0.8072 - val_loss: 0.3951 - val_accuracy: 0.8293
Epoch 4/50
27/27 [==============================] - 0s 4ms/step - loss: 0.4013 - accuracy: 0.7956 - val_loss: 0.4055 - val_accuracy: 0.8157
Epoch 5/50
27/27 [==============================] - 0s 3ms/step - loss: 0.3962 - accuracy: 0.8165 - val_loss: 0.4057 - val_accuracy: 0.8022
Epoch 6/50
27/27 [==============================] - 0s 3ms/step - loss: 0.3920 - accuracy: 0.8060 - val_loss: 0.3989 - val_accuracy: 0.8130
Epoch 7/50
27/27 [==============================] - 0s 3ms/step - loss: 0.3847 - accuracy: 0.8188 - val_loss: 0.4306 - val_accuracy: 0.8022
Epoch 8/50
27/27 [==

In [22]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.44880977272987366, 0.7696477174758911]

In [24]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.45
accuracy: 76.96


In [25]:
# Predictions are probabilities.

predictions = model.predict(X_test)

12/12 [==============================] - 0s 955us/step


In [27]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)[:10] # show first 10 predictions

array([[1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [28]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, np.round(predictions))

array([[124,  44],
       [ 41, 160]])

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [29]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])

In [30]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data = (X_test, y_test), 
    callbacks=[early_stop])  # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 2s 16ms/step - loss: 0.4821 - accuracy: 0.7538 - val_loss: 0.4037 - val_accuracy: 0.8184
Epoch 2/20
27/27 [==============================] - 0s 8ms/step - loss: 0.4247 - accuracy: 0.8026 - val_loss: 0.3863 - val_accuracy: 0.8266
Epoch 3/20
27/27 [==============================] - 0s 8ms/step - loss: 0.4216 - accuracy: 0.7909 - val_loss: 0.3977 - val_accuracy: 0.8293
Epoch 4/20
27/27 [==============================] - 0s 9ms/step - loss: 0.4200 - accuracy: 0.7863 - val_loss: 0.3891 - val_accuracy: 0.8211
Epoch 5/20
27/27 [==============================] - 0s 9ms/step - loss: 0.4038 - accuracy: 0.8026 - val_loss: 0.4145 - val_accuracy: 0.7940
Epoch 6/20
27/27 [==============================] - 0s 9ms/step - loss: 0.3820 - accuracy: 0.8211 - val_loss: 0.3909 - val_accuracy: 0.8211
Epoch 7/20
27/27 [==============================] - 0s 9ms/step - loss: 0.3813 - accuracy: 0.8235 - val_loss: 0.3941 - val_accuracy: 0.8293
Epoch 7: early stop

In [31]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores
# In results, first is loss, second is accuracy

[0.3940976858139038, 0.8292682766914368]

In [32]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.39
accuracy: 82.93


# LSTM with one layer

In [33]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]) # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 1s 13ms/step - loss: 0.4449 - accuracy: 0.7816 - val_loss: 0.3857 - val_accuracy: 0.8320
Epoch 2/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4190 - accuracy: 0.7909 - val_loss: 0.3826 - val_accuracy: 0.8401
Epoch 3/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4172 - accuracy: 0.8002 - val_loss: 0.3812 - val_accuracy: 0.8401
Epoch 4/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4081 - accuracy: 0.7933 - val_loss: 0.3871 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4094 - accuracy: 0.8014 - val_loss: 0.3932 - val_accuracy: 0.8130
Epoch 6/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4080 - accuracy: 0.7967 - val_loss: 0.3843 - val_accuracy: 0.8347
Epoch 7/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4071 - accuracy: 0.8002 - val_loss: 0.3832 - val_accuracy: 0.8320
Epoch 8/20
27/27 [=

In [35]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.3908209800720215, 0.8102980852127075]

In [36]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.39
accuracy: 81.03


# LSTM with more layers

In [37]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 3s 35ms/step - loss: 0.5068 - accuracy: 0.7607 - val_loss: 0.3924 - val_accuracy: 0.8347
Epoch 2/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4362 - accuracy: 0.7840 - val_loss: 0.3807 - val_accuracy: 0.8266
Epoch 3/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4321 - accuracy: 0.7944 - val_loss: 0.3883 - val_accuracy: 0.8266
Epoch 4/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4166 - accuracy: 0.7944 - val_loss: 0.3892 - val_accuracy: 0.8130
Epoch 5/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4144 - accuracy: 0.8060 - val_loss: 0.3969 - val_accuracy: 0.8238
Epoch 6/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4162 - accuracy: 0.7933 - val_loss: 0.3847 - val_accuracy: 0.8293
Epoch 7/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4104 - accuracy: 0.7967 - val_loss: 0.4013 - val_accuracy: 0.8076
Epoch 7: earl

In [39]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4012657403945923, 0.8075881004333496]

In [40]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.40
accuracy: 80.76%


# GRU with one layer

In [41]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 1s 14ms/step - loss: 0.4701 - accuracy: 0.7747 - val_loss: 0.3872 - val_accuracy: 0.8293
Epoch 2/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4159 - accuracy: 0.7863 - val_loss: 0.3884 - val_accuracy: 0.8347
Epoch 3/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4151 - accuracy: 0.7944 - val_loss: 0.3870 - val_accuracy: 0.8347
Epoch 4/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4062 - accuracy: 0.7909 - val_loss: 0.3995 - val_accuracy: 0.8076
Epoch 5/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4079 - accuracy: 0.8072 - val_loss: 0.3952 - val_accuracy: 0.8266
Epoch 6/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4041 - accuracy: 0.7933 - val_loss: 0.3931 - val_accuracy: 0.8347
Epoch 7/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4000 - accuracy: 0.7991 - val_loss: 0.3929 - val_accuracy: 0.8293
Epoch 8/20
27/27 [=

In [43]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.403528094291687, 0.7994579672813416]

In [44]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.40
accuracy: 79.95


# GRU with more layers

In [45]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [46]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 3s 35ms/step - loss: 0.4708 - accuracy: 0.7526 - val_loss: 0.3807 - val_accuracy: 0.8266
Epoch 2/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4231 - accuracy: 0.7944 - val_loss: 0.3844 - val_accuracy: 0.8428
Epoch 3/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4282 - accuracy: 0.7875 - val_loss: 0.4050 - val_accuracy: 0.8130
Epoch 4/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4131 - accuracy: 0.7956 - val_loss: 0.4117 - val_accuracy: 0.7940
Epoch 5/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4176 - accuracy: 0.7967 - val_loss: 0.4022 - val_accuracy: 0.8293
Epoch 6/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4114 - accuracy: 0.8072 - val_loss: 0.3978 - val_accuracy: 0.8211
Epoch 6: early stopping


In [47]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.3977782726287842, 0.8211382031440735]

In [48]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.40
accuracy: 82.11%


# Conv1D

In [49]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 10)            40        
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 20)            620       
                                                                 
 lstm_4 (LSTM)               (None, 13, 32)            6784      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15,797
Trainable params: 15,797
Non-trainable params: 0
_________________________________________________________________


In [50]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 2s 19ms/step - loss: 0.5416 - accuracy: 0.7224 - val_loss: 0.4066 - val_accuracy: 0.8509
Epoch 2/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4634 - accuracy: 0.7666 - val_loss: 0.3837 - val_accuracy: 0.8022
Epoch 3/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4688 - accuracy: 0.7584 - val_loss: 0.3787 - val_accuracy: 0.8238
Epoch 4/20
27/27 [==============================] - 0s 6ms/step - loss: 0.4471 - accuracy: 0.7805 - val_loss: 0.4102 - val_accuracy: 0.8157
Epoch 5/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4359 - accuracy: 0.7758 - val_loss: 0.3834 - val_accuracy: 0.8049
Epoch 6/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4237 - accuracy: 0.7967 - val_loss: 0.3806 - val_accuracy: 0.8293
Epoch 7/20
27/27 [==============================] - 0s 7ms/step - loss: 0.4135 - accuracy: 0.8095 - val_loss: 0.3846 - val_accuracy: 0.8266
Epoch 8/20
27/27 [=

In [51]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.38204607367515564, 0.8184281587600708]

In [52]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.38
accuracy: 81.84
